<a href="https://colab.research.google.com/github/kyokicchi/ggl_an_codes/blob/master/trainLGBmodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install lightgbm

    100% |████████████████████████████████| 1.1MB 8.7MB/s 


In [0]:
import lightgbm as lgb
import numpy as np
import pandas as pd
from google.colab import drive
from google.colab import files
from sklearn.model_selection import train_test_split
import pickle

In [3]:
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [4]:
%cd ../content/gdrive/My Drive/projects/ggl_an/
%ls

/content/gdrive/My Drive/projects/ggl_an
user_train.csv


In [0]:
s_data = 'user_train.csv'
filename = 'model_LGB.sav'
s_col_tgt = 'transactionRevenue'
l_col_drop = [s_col_tgt]

In [6]:
%%time
df_train = pd.read_csv(s_data, index_col = 0)

CPU times: user 1.83 s, sys: 313 ms, total: 2.14 s
Wall time: 2.99 s


<string>:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.


In [7]:
%%time
df_y = df_train[s_col_tgt]
df_x = df_train.drop(l_col_drop, axis=1)

CPU times: user 25.3 ms, sys: 68.7 ms, total: 94.1 ms
Wall time: 102 ms


In [13]:
%%time
x_train, x_test, y_train, y_test = train_test_split(df_x, df_y)



CPU times: user 405 ms, sys: 987 µs, total: 406 ms
Wall time: 408 ms


In [14]:
%%time
from sklearn.model_selection import GridSearchCV

def applyGSCV(model, param, X, Y):
    res = GridSearchCV(model, param, cv=3)
    res.fit(X, Y)
    return res


model = lgb.LGBMRegressor()

param = {
    'learning_rate': [0.1,0.15],
    'n_estimators': [200,500],
    'num_leaves': [50,100],
}
  
model_LGB = applyGSCV(model, param, x_train, y_train)



CPU times: user 19min 39s, sys: 7.2 s, total: 19min 46s
Wall time: 10min 6s


In [15]:
print(model_LGB.best_score_)
print(model_LGB.best_estimator_)


0.3889117907976865
LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
       importance_type='split', learning_rate=0.1, max_depth=-1,
       min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
       n_estimators=200, n_jobs=-1, num_leaves=50, objective=None,
       random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
       subsample=1.0, subsample_for_bin=200000, subsample_freq=0)


In [17]:
%%time
model = lgb.LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
       importance_type='split', learning_rate=0.1, max_depth=-1,
       min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
       n_estimators=400, n_jobs=-1, num_leaves=50, objective=None,
       random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
       subsample=1.0, subsample_for_bin=200000, subsample_freq=0)
model.fit(x_train, 
          y_train,
          eval_set=[(x_train, y_train),(x_test, y_test)],
          eval_metric='rmse',
          early_stopping_rounds = 300)

[1]	training's rmse: 2.83232	training's l2: 8.02202	valid_1's rmse: 2.60984	valid_1's l2: 6.81128
Training until validation scores don't improve for 300 rounds.
[2]	training's rmse: 2.7301	training's l2: 7.45346	valid_1's rmse: 2.52042	valid_1's l2: 6.35252
[3]	training's rmse: 2.64298	training's l2: 6.98533	valid_1's rmse: 2.44489	valid_1's l2: 5.97748
[4]	training's rmse: 2.57228	training's l2: 6.61664	valid_1's rmse: 2.38232	valid_1's l2: 5.67545
[5]	training's rmse: 2.51114	training's l2: 6.3058	valid_1's rmse: 2.32704	valid_1's l2: 5.41509
[6]	training's rmse: 2.45638	training's l2: 6.03379	valid_1's rmse: 2.27869	valid_1's l2: 5.19242
[7]	training's rmse: 2.40916	training's l2: 5.80403	valid_1's rmse: 2.23959	valid_1's l2: 5.01577
[8]	training's rmse: 2.37198	training's l2: 5.62627	valid_1's rmse: 2.20783	valid_1's l2: 4.87449
[9]	training's rmse: 2.33979	training's l2: 5.47462	valid_1's rmse: 2.17909	valid_1's l2: 4.74845
[10]	training's rmse: 2.31098	training's l2: 5.34063	vali

In [18]:
%%time
pickle.dump(model_LGB, open(filename, 'wb'))

# files.download(filename) 

CPU times: user 54.5 ms, sys: 1.96 ms, total: 56.4 ms
Wall time: 44.5 ms
